#  BKLM Data Taking - Short Guide

you can find the original Page over [here](https://confluence.desy.de/display/BI/BKLM+Data+Taking+-+Short+Guide).


![VNC Setup](img/kek_vnc.png)


For the time beeing the Controlling of the HV is ignored. 

The setup uses the following Computers/shells.

|Shell |      Computer  |  Use |
|----------|-----------|------|
|KLM01_A   | klm01       | Start/Kill run |
|KLM01_B   | klm01       |   Auxiliary terminal to debug COPPERs |
|ttd11_A   | ttd11       |    Program FEEs; Include / mask sectors; Start reset triggers  |
|IDLAB_A   | idlab       | configure BF2   |
|ttd11_B   | ttd11       | Local run status |
|ttd11_C   | ttd11       | Global run status| 
|KLM01_C   | klm01       | Check COPPERS status|
|KLM01_D   | klm01       | Launch SLC     | 




In [1]:
import  config.config_kek as  config
from Linux_Helpers.remoteshell2 import remoteShell2
from Linux_Helpers.remoteshell3 import remoteShell3

#from equipment.ScintilatorReadout import ScintilatorReadout
from equipment.py_ftsw import py_ftsw
from script.prog_fee import prog_fee
from script.prepft import prepft
import time

In [2]:
from config.load_known_hosts import load_known_hosts
from Linux_Helpers.ssh_host import shell_factory


In [3]:
from config.RCL_Load_Config import RCL_Load_Config


In [4]:
from config.ftsw_load_config import ftsw_load_config
from equipment.py_ftsw import create_ftsw


In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

from ipywidgets import widgets
import ipython_blocking
ipython_blocking.load_ipython_extensions()

from IPython.display import clear_output




## Setting up the envirement

Since we want to use remote terminals, we have to set them up. Since this notebook is not (yet) connected to the KEK setup all the shells are just dummies and most likley they will produce some errors.

In [6]:
file_name="config/known_host_KEK.xlsx"
KLM_Conf = "BFR_a_Not_working"

Known_hosts = load_known_hosts(file_name)
Sh_Factory = shell_factory(Known_hosts)
ftsw_conf = ftsw_load_config(file_name,KLM_Conf)

In [7]:
ftsw = create_ftsw(Sh_Factory ,ftsw_conf )

KLM01_A = Sh_Factory.get_shell('klm01')


class klm01:
    HostName = "127.0.0.1"
    UserName = "b2klm"
    IdentityFile = "config/b2klm"
    Port = 10040
    startCommand = ""
    endstring = "b2klm.klm@klm01"
    
    
    
KLM01_B = remoteShell3(host=klm01)

IDLAB_A = Sh_Factory.get_shell('idlab')


In [8]:
print(KLM01_B.sendAndRecieveRaw('ls',removePromt=False))



20150512.0507.dat             KEKFW/               run/
%backup%~*                    klm02.conf           run_ctrl7_OCT3.dat
basf2@                        klm02_weKLM.conf     scripts/
basf2_local/                  klm02_weKLM.conf~    setup.sh*
batchTesths*                  klm03.conf           SOCK_KLM01_klm01.txt
Belle2FileCatalog.xml         #klm.conf#           src/
bin/                          klm.conf             t3.txt
bklmEffi.root                 klm.conf~            temp/
daq_slc@                      klmslscan/           temp1.conf
daq_slc.copper@               log/                 temp2.conf
daq_slc-sl5/                  log180215-15:45      temp.conf
daq_slc-sl6/                  log180216-09:00      temp.conf~
diagnos_output.root           nsmd2.1970*          test2.txt
eb/                           o.txt                tmp/
epics/                        Output.sroot         tmp3.sroot
externals/                    rcl_rev2_10.zip*     trg_scan_setgain_dbg.txt
fefw4/

In [9]:
def f(DebugPrint):
    ftsw.debug(DebugPrint)
    
x=interact(f,DebugPrint=False)

interactive(children=(Checkbox(value=False, description='DebugPrint'), Output()), _dom_classes=('widget-intera…

In [10]:
# don't go beyond here with Run All
assert False

AssertionError: 

Once the Notebook is connected to KEK, 
all of these commands should run without error.
If the command `remoteShell2` produces and error make sure the SSH_Tunnel is setup correctly and to PocketDAQ / Copper is running

# Preliminary steps


## Program the FEEs

- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following commands:
```
./progfee.sh
./progfee_BB.sh
```


To try new firmware
- upload your bitfile into the `Firmware/` directory 
- modify the variable(s) `SCROD_firmware` and/or `DC_firmware` in the cell(s) below
- execute the cell(s) below

In [15]:
from config.Dev_details_load import Dev_details_load
BB = Dev_details_load(file_name,KLM_Conf,'BB')
BF = Dev_details_load(file_name,KLM_Conf,'BB')

In [14]:
SCROD_firmware = "181017_klmscint_simple_top.bit"
SCROD_FW_source = "Firmware/" + SCROD_firmware
SCROD_FW_target = "lastfw/" + SCROD_firmware
ftsw.shell.copyTo(SCROD_FW_source, SCROD_FW_target)


In [64]:
DC_firmware = "concentrator_top_revo_frame9_ec6295315a4.bit"
DC_FW_source = "Firmware/" + DC_firmware
DC_FW_target = "lastfw/" + DC_firmware
ftsw.shell.copyTo(DC_FW_source, DC_FW_target)

~/lastfw/concentrator_top_revo_frame9_ec6295315a4.bit


In [16]:
BB.scrod_firmware = SCROD_FW_target
BF.scrod_firmware = SCROD_FW_target

lastfw/181017_klmscint_simple_top.bit



In [ ]:
BB.dataConcentrator_Firmware = DC_firmware
BF.dataConcentrator_Firmware = DC_firmware

In [17]:
print(prog_fee(ftsw,BB))

jtag s103 (feeid=16606)
jtag s107 (feeid=16608)
jtag s108 (feeid=16703)
jtag s109 (feeid=16705)
jtag s110 (feeid=16706)
jtag s112 (feeid=16708)
jtag s095 (feeid=16803)
jtag s097 (feeid=16805)
jtag s094 (feeid=16806)
jtag s093 (feeid=16808)
jtag s102 (feeid=16903)
jtag s098 (feeid=16905)
jtag s099 (feeid=16906)
jtag s101 (feeid=16908)
jtagft version 2018011600
Number of devices = 1
************* TDO(head) *************
* DEV[0]: xc6slx150t (0403d093)
************* TDI(tail) *************
jtagft version 2018011600
device=6slx150tfgg676
idcode = 4403d093
size = 4222164 211108
status = 035 (done) 00003cec
done.
jtag s099 (feeid=16906)
jtagft version 2018011600
Number of devices = 1
************* TDO(head) *************
* DEV[0]: xc6slx150t (0403d093)
************* TDI(tail) *************
jtagft version 2018011600
device=6slx150tfgg676
idcode = 4403d093
size = 4222164 211108
status = 035 (done) 00003cec
done.
jtag dc24 (feeid=16607)
jtag dc25 (feeid=16704)
jtag dc26 (feeid=16707)
jtag dc27 

In [18]:
print(prog_fee(ftsw,BF))

jtag s103 (feeid=16606)
jtag s107 (feeid=16608)
jtag s108 (feeid=16703)
jtag s109 (feeid=16705)
jtag s110 (feeid=16706)
jtag s112 (feeid=16708)
jtag s095 (feeid=16803)
jtag s097 (feeid=16805)
jtag s094 (feeid=16806)
jtag s093 (feeid=16808)
jtag s102 (feeid=16903)
jtag s098 (feeid=16905)
jtag s099 (feeid=16906)
jtag s101 (feeid=16908)
jtagft version 2018011600
Number of devices = 1
************* TDO(head) *************
* DEV[0]: xc6slx150t (0403d093)
************* TDI(tail) *************
jtagft version 2018011600
device=6slx150tfgg676
idcode = 4403d093
size = 4222164 211108
status = 035 (done) 00003cec
done.
jtag s099 (feeid=16906)
jtagft version 2018011600
Number of devices = 1
************* TDO(head) *************
* DEV[0]: xc6slx150t (0403d093)
************* TDI(tail) *************
jtagft version 2018011600
device=6slx150tfgg676
idcode = 4403d093
size = 4222164 211108
status = 035 (done) 00003cec
done.
jtag dc24 (feeid=16607)
jtag dc25 (feeid=16704)
jtag dc26 (feeid=16707)
jtag dc27 

possible errors 

```
too many devices found in the jtag chain (probably noise).
```


- Check: the last messages shown in the terminal are:

   `ALL KLM FEE PROG DONE!` 
   
   and 
   
   `ALL KLM BB FEE PROG DONE!` 
   
   In the meanwhile, <span style="color:purple">the bottom-center terminal, left tab (`ttd11_B`) should be in an ERROR  state and become purple</span>: it's fine.
   
In a future modification to this Notebook the status of `ttd11_B` should actually be checked.


## Program the BF2 sector

(Note added on 2018-05-01: BF2 is currently masked, so the commands this section can be skipped- Now BB2 is acting as BF2 for debug interface, but script names are the same for ease of use)


- Where? `IDLAB_A` (bottom-left terminal, right tab)
- How? launch the following commands:

```
  impact -batch imp_prog_BF2scrods.cmd
  impact -batch imp_prog_BF2dc.cmd
```




To change the version of the firmware that is being programmed to SCROD FPGAs:
- move your bitfile to the `Firmware` directory on this virtual machine
- specify the name of your bitfile under `SCROD_firmware` in the next cell
- execute the next four cells



In [19]:
SCROD_firmware = "181017_klmscint_simple_top.bit"
FW_path = "/home/idlab/lastfw/"
BFIDLAB = FW_path+SCROD_firmware
IDLAB_A.copyTo("Firmware/"+SCROD_firmware, BFIDLAB)

In [20]:
impactCommand = "imp_prog_BF2scrods_latest.cmd"

In [21]:
def CreateImpactCommandFile (bitFile, cmdFile):
    # Batch command for programming SCRODs via command line
    outCmdFile = 'setMode -bs \nsetMode -bs \nsetMode -bs \nsetMode -bs \nsetCable -target "digilent_plugin DEVICE=SN:210249992992 FREQUENCY=10000000" \nIdentify -inferir  \nidentifyMPM  \nassignFile -p 1 -file "' + bitFile + '"\nProgram -p 1  \nsetCable -target "digilent_plugin DEVICE=SN:210249993114 FREQUENCY=10000000" \nIdentify -inferir  \nidentifyMPM  \nassignFile -p 1 -file "' + bitFile + '"\nProgram -p 1  \nsetMode -bs \nsetMode -bs \ndeleteDevice -position 1 \nsetMode -bs \nsetMode -ss \nsetMode -sm \nsetMode -hw140 \nsetMode -spi \nsetMode -acecf \nsetMode -acempm \nsetMode -pff'
    print(outCmdFile)
    with open(cmdFile, 'w') as f:
        f.write(outCmdFile)

In [22]:
CreateImpactCommandFile(BFIDLAB, impactCommand)
IDLAB_A.copyTo(impactCommand, impactCommand)


setMode -bs 
setMode -bs 
setMode -bs 
setMode -bs 
setCable -target "digilent_plugin DEVICE=SN:210249992992 FREQUENCY=10000000" 
Identify -inferir  
identifyMPM  
assignFile -p 1 -file "/home/idlab/lastfw/181017_klmscint_simple_top.bit"
Program -p 1  
setCable -target "digilent_plugin DEVICE=SN:210249993114 FREQUENCY=10000000" 
Identify -inferir  
identifyMPM  
assignFile -p 1 -file "/home/idlab/lastfw/181017_klmscint_simple_top.bit"
Program -p 1  
setMode -bs 
setMode -bs 
deleteDevice -position 1 
setMode -bs 
setMode -ss 
setMode -sm 
setMode -hw140 
setMode -spi 
setMode -acecf 
setMode -acempm 
setMode -pff


In [23]:
print(IDLAB_A.sendAndRecieve("source bashrc-ise.sh;impact -batch " + impactCommand))

. /opt/Xilinx/14.7/ISE_DS/common/.settings64.sh /opt/Xilinx/14.7/ISE_DS/common
. /opt/Xilinx/14.7/ISE_DS/EDK/.settings64.sh /opt/Xilinx/14.7/ISE_DS/EDK
. /opt/Xilinx/14.7/ISE_DS/PlanAhead/.settings64.sh /opt/Xilinx/14.7/ISE_DS/PlanAhead
. /opt/Xilinx/14.7/ISE_DS/ISE/.settings64.sh /opt/Xilinx/14.7/ISE_DS/ISE
Release 14.7 - iMPACT P.20131013 (lin64)
Copyright (c) 1995-2013 Xilinx, Inc.  All rights reserved.
Preference Table
Name                 Setting             
StartupClock         Auto_Correction     
AutoSignature        False               
KeepSVF              False               
ConcurrentMode       False               
UseHighz             False               
ConfigOnFailure      Stop                
UserLevel            Novice              
MessageLevel         Detailed            
svfUseTime           false               
SpiByteSwap          Auto_Correction     
AutoInfer            false               
SvfPlayDisplayComments false               
Identifying chain content

In [24]:

print(IDLAB_A.sendAndRecieve("source bashrc-ise.sh;impact -batch imp_prog_BF2dc.cmd"))

. /opt/Xilinx/14.7/ISE_DS/common/.settings64.sh /opt/Xilinx/14.7/ISE_DS/common
. /opt/Xilinx/14.7/ISE_DS/EDK/.settings64.sh /opt/Xilinx/14.7/ISE_DS/EDK
. /opt/Xilinx/14.7/ISE_DS/PlanAhead/.settings64.sh /opt/Xilinx/14.7/ISE_DS/PlanAhead
. /opt/Xilinx/14.7/ISE_DS/ISE/.settings64.sh /opt/Xilinx/14.7/ISE_DS/ISE
Release 14.7 - iMPACT P.20131013 (lin64)
Copyright (c) 1995-2013 Xilinx, Inc.  All rights reserved.
Preference Table
Name                 Setting             
StartupClock         Auto_Correction     
AutoSignature        False               
KeepSVF              False               
ConcurrentMode       False               
UseHighz             False               
ConfigOnFailure      Stop                
UserLevel            Novice              
MessageLevel         Detailed            
svfUseTime           false               
SpiByteSwap          Auto_Correction     
AutoInfer            false               
SvfPlayDisplayComments false               
Connecting to cable (Usb 

In [13]:
#print(IDLAB_A.sendAndRecieve("ls /home/idlab/"))

- Check: one of the last message shown in the terminal is:

```
INFO:iMPACT - '1': Checking done pin....done.   / /   '1': Programmed successfully.   / /   Elapsed time =  X sec.
```


  Ignore further error messages!  
  
  
  ```
  INFO:iMPACT - '1': Checking done pin....done.
'1': Programmed successfully.
Elapsed time =     10 sec.
ERROR:iMPACT - SSerial mode is no longer supported by impact software , please
   use 11.4 or older version of software.
```



## Prepare ttd11
- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following commands:
```        
  resetft -191
```
  With EKLM: 
```  
cd withEKLM; ./prepft_butBF2_withEKLM.sh
```

   Without EKLM: 
```
./prepft_butBF2.sh
```




In [25]:
useEKLM = False
if useEKLM:
    print(prepft(ftsw,config.klm_prepft_with_EKLM))
else:
    print(prepft(ftsw,BB))


trigft version 2018021900
resetting trigger
utimeft: utime is set, n=10565198 sec=1541382724 usec=0
use s117 (feeid=16206)
use s118 (feeid=16208)
use s156 (feeid=16303)
use s157 (feeid=16305)
use s158 (feeid=16306)
use s159 (feeid=16308)
use s051 (feeid=16403)
use s075 (feeid=16405)
use s092 (feeid=16406)
use s104 (feeid=16408)
use s150 (feeid=16503)
use s152 (feeid=16505)
use s160 (feeid=16506)
use s161 (feeid=16508)
use s154 (feeid=16203)
use s153 (feeid=16205)
use dc12 (feeid=16507)
use dc13 (feeid=16504)
use dc16 (feeid=16207)
use dc17 (feeid=16307)
use dc18 (feeid=16304)
use dc19 (feeid=16404)
use dc20 (feeid=16407)
use dc46 (feeid=16604)
use s103 (feeid=16606)
use s107 (feeid=16608)
use s108 (feeid=16703)
use s109 (feeid=16705)
use s110 (feeid=16706)
use s112 (feeid=16708)
use s095 (feeid=16803)
use s097 (feeid=16805)
use s094 (feeid=16806)
use s093 (feeid=16808)
use s102 (feeid=16903)
use s098 (feeid=16905)
use s099 (feeid=16906)
use s101 (feeid=16908)
use dc24 (feeid=16607)
use

with EKLM not yet tested

```python
useEKLM = False
if useEKLM:
    print(ttd11_A.sendAndRecieve("cd withEKLM; ./prepft_butBF2_withEKLM.sh"))
else:
    print(ttd11_A.sendAndRecieve("./prepft_butBF2.sh"))
```



- Check: 
  
  the last message shown in the terminal is:
```
resetting trigger
```

## Clear "b2linkdown" after reprograming

Usually you will see `b2linkdown` after reprograming, `b2linkdown` shown in the the bottom-center terminal. Run the following:


- At `KLM01_B` run:
```
~/run/scripts/batchTesths  
```

  `batchTesths` does "tesths -abcd" for all four coppers)


- At `ttd11_A` run:
```
./prepft_butBF2.sh
```

In [54]:
print(Sh_Factory.run_on('cpr7001',"tesths -abcd"))

tesths: version 0.21 slot A B C D
hslb-A Belle2link is down, csr=18000003
hslb-B 0.67 18001100
hslb-C 0.67 18001100
hslb-D 0.67 18001100



In [27]:
print(Sh_Factory.run_on('cpr7002',"tesths -abcd"))

tesths: version 0.21 slot A B C D
hslb-A 0.67 18001000
hslb-B 0.67 18001000
hslb-C 0.67 18001000
hslb-D 0.67 18001000



In [28]:
print(Sh_Factory.run_on('cpr7003',"tesths -abcd"))

tesths: version 0.21 slot A B C D
hslb-A Belle2link recovered, csr=18001114 (retry 1)
hslb-A 0.67 18001114
hslb-B 0.67 18001010
hslb-C 0.67 18001110
hslb-D 0.67 18001010



In [29]:
print(Sh_Factory.run_on('cpr7004',"tesths -abcd"))

tesths: version 0.21 slot A B C D
hslb-A 0.67 18001110
hslb-B 0.67 18001110
hslb-C 0.67 18001110
hslb-D 0.67 18001110



In [31]:
print(prepft(ftsw,config.klm_prepft))

trigft version 2018021900
resetting trigger
utimeft: utime is set, n=3232001 sec=1541382939 usec=0
use dc16 (feeid=16207)
use s117 (feeid=16206)
use s118 (feeid=16208)
use dc17 (feeid=16307)
use s156 (feeid=16303)
use s157 (feeid=16305)
use dc18 (feeid=16304)
use s158 (feeid=16306)
use s159 (feeid=16308)
use dc12 (feeid=16507)
use s160 (feeid=16506)
use s161 (feeid=16508)
use dc19 (feeid=16404)
use s051 (feeid=16403)
use s075 (feeid=16405)
use dc20 (feeid=16407)
use s092 (feeid=16406)
use s104 (feeid=16408)
use dc13 (feeid=16504)
use s150 (feeid=16503)
use s152 (feeid=16505)
use dc46 (feeid=16604)
use dc24 (feeid=16607)
use dc25 (feeid=16704)
use dc26 (feeid=16707)
use s113 (feeid=16603)
use s116 (feeid=16605)
use s103 (feeid=16606)
use s107 (feeid=16608)
use s108 (feeid=16703)
use s109 (feeid=16705)
use s110 (feeid=16706)
use s112 (feeid=16708)
use dc27 (feeid=16804)
use dc29 (feeid=16807)
use dc21 (feeid=16904)
use dc22 (feeid=16901)
use s095 (feeid=16803)
use s097 (feeid=16805)
use 

In [32]:
print(ftsw.status())

statft-20180821 FTSW #191 / ft3p068a 2018.08.06-13:20:24 -> 11.05 10:55:47
16  exprun=00010100 exp 0 run 257 sub 0
17   omask=00009fbc s3q=0 clk=00 lmask=1fbc LOCAL
1f    jpll=cc008000 clk=in GOOD-CLOCK
28292c trg=00000000 none limit 0 <-> last 0
2a2b27 cnt 0 > 0 > 0 > 0 (0.0 > 0.0 > 0.0Hz)
2d stafifo=10000000 empty trg-DISABLED
20   reset=80000000 11.05-10:55:39.922(reset) no-FIFO 
31     err=80000000 11.05-10:55:39.922(error)
25/30 e/bs=0f000000 01000000
393a3b  me=19100004 0f000000 10800043 min=610
405468  O0=22900000 0a000000 1080000f ready tag=0 min=3..0 d=0.00%
415569  O1=23000000 0a000000 1080000f ready tag=0 min=3..0 d=0.00%
45596d  o5!24500000 0f000000 1080001f - LOCAL-mode d=0.00%
465a6e  X6=17080000 0a000000 00abcdef ready tag=0 d=0.00%
9f limiter=01000100 maxtrig=1 maxtime=2.00us
a0-a7 dead 0.00% (t=0.00% c=0.00% p=0.00% f=0.00% r=0.00%)



then you should able to see `b2linkdown` was cleared, <span style="color:gold">and `FTSW` in yellow READY </span>state.

# Launch the Slow Control

- Where? `KLM01_D` (bottom-right terminal, right tab)
- How? launch the following commands:


```
    ~/run/scripts/load_brcl.sh rcl_rev2_40_th-100  
```

    
    (NOTE: Every time you send this command, you need to resend the following commands)

    NOTE: kill all that `watch` commands which are reading status of registers to avoid any conflict when you issue the load_brcl.sh. Kill the 
  
  ```
    watch ./checkpcr 
```
  
    and 
  
  ```
    watch ./dcregs.sh
```

    Don't keep `watch ./dcregs.sh` running! Unless after reprograming you want to check the status, you run

```
    watch ./dcreg.sh
```

    then kill it right away otherwise you will forget later.  

In [55]:
from script.load_brcl import load_brcl
from script.load_klm_calset import load_klm_calset
RCL = RCL_Load_Config(file_name,KLM_Conf)

In [56]:
load_brcl(shell_factory=Sh_Factory,RCL=RCL,Parallel=True)

BFL_B Fail
Successfull configured BBL_C
window coarse start: 768
window coarse stop: 332
window fine start: 144
window fine stop: 16

Successfull configured BFL_A
Successfull configured BFL_B
window coarse start: 768
window coarse stop: 332
window fine start: 144
window fine stop: 16

Successfull configured BBL_A
Successfull configured BBR_D
window coarse start: 768
window coarse stop: 332
window fine start: 144
window fine stop: 16

Successfull configured BBR_A
Successfull configured BFL_D
Successfull configured BBL_D
Successfull configured BBL_B
Successfull configured BBR_B
Successfull configured BBR_C
Successfull configured BFL_B


''

- Check: the last message shown in the terminal is:

`window fine stop   / /   reg0033 = XXXXXXXX`

`Successfull configured ...`


- Then run this command to send MPPC gain settings to Layer 1 on Barrel - wait for all windows to finish completely before launching next one
```
cd ~/run/scripts
./load_klm_calset_L1.sh calset_171118_031700 200
```

In [57]:
load_klm_calset(Sh_Factory,RCL,True)

/home/group/b2klm/run/scripts/calset_171118_031700/scpr7002aL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7002aL2.sh  200
/home/group/b2klm/run/scripts/calset_171118_031700/scpr7002bL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7002bL2.sh  200
/home/group/b2klm/run/scripts/calset_171118_031700/scpr7002cL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7002cL2.sh  200
/home/group/b2klm/run/scripts/calset_171118_031700/scpr7002dL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7002dL2.sh  200
/home/group/b2klm/run/scripts/calset_171118_031700/scpr7003aL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7003aL2.sh  200
/home/group/b2klm/run/scripts/calset_171118_031700/scpr7003bL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7003bL2.sh  200
/home/group/b2klm/run/scripts/calset_171118_031700/scpr7003cL1.sh  200
/home/group/b2klm/run/scripts/calset_171118_060700/scpr7003cL2.sh  200
/home/

- Wait for command to finish (it opens 16 concurrent terminals, so need to have X tunneling)

- after this run the following command:

    `./load_klm_calset_L2.sh calset_171118_060700 200`

- wait for previous command to completely finish, then set lookback and width for scintilaltor 

- make sure you do not use any other scint. lookback commands after this:
- `./set_addl.sh` (this command is very quiet) (if you see "Device or resource busy" message, just run this command again.)


In [58]:
print(Sh_Factory.run_on('klm01',"cd ~/run/scripts; ./set_addl.sh"))

## Reset the trigger
- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following command:

```
resetft -191
```


In [59]:
ftsw.reset()

'trigft version 2018021900\nresetting trigger\n'

- Check: the bottom-center terminal, left tab <span style="color:gold">`ttd11_B` should pass in a READY state and become yellow </span>

# Start a run


## Prepare basf2:
- Where? `@klm01` (top-left terminal, left tab)
- How? launch the following command:
  - With EKLM: 
  ```bash
  cd ~/basf2/release/daq/copper/daq_scripts/beklm; ./run_start_mono_klm_butBF2.sh
  ```
  - Without EKLM:
  ```bash
  ./run_start_mono_Barrel_butBF2.sh
  ```
- Check: eight little windows should appear in the display, the top four windows should all show "ffffffff" at tail.
- If you re-login klm01 as b2klm, need to setup basf2 environment, do the following (you can type "basf" then tab to see whether basf2 command is available, if it is not, it means basf2 environment is not set)
```bash
source ~b2klm/basf2/tools/setup_belle2
cd ~b2klm/basf2/release/; setuprel
cd ~/basf2/release/daq/copper/daq_scripts
``` 


### Setup basf2 environment and release
- typical output:
```
'cd /home/group/b2klm/basf2_local/release/daq/copper/daq_scripts\r'
```

In [41]:
print(KLM01_B.sendAndRecieveRaw("cd /home/group/b2klm/basf2_local/tools_161125/; source setup_belle2"))



Belle II software tools set up at: /home/group/b2klm/basf2_local/tools_161125


In [42]:
print(KLM01_B.sendAndRecieveRaw("cd /home/group/b2klm/basf2_local/release/; setuprel"))


From ssh://stash.desy.de:7999/b2/tools
   ae235f7..0677d45  master     -> origin/master

Environment setup for release: head
Local release directory      : /home/group/b2klm/basf2_local/release



In [43]:
print(KLM01_B.sendAndRecieveRaw("cd /home/group/b2klm/basf2_local/release/daq/copper/daq_scripts"))

### Designate COPPER nodes and readout PCs for data acquisition



In [45]:
des_ser_COPPER_main = "/home/group/b2klm/basf2_local/release/daq/ropc/des_ser_COPPER_main"
des_ser_ROPC_main   = "/home/group/b2klm/basf2_local/release/daq/ropc/des_ser_ROPC_main"

FINNESEbitflag  = ["14","15","15","15"]
COPPER_hostname = [0 for i in range(4)]
COPPER_nodeID   = [0 for i in range(4)]
eb0_port        = [0 for i in range(4)]
COPPER_shell    = [0 for i in range(4)]

UseNSM_bool       = str(0)
NSMnodeName_cpr   = str(1)
NSMnodeName_ropc  = 'hogehoge1000'

for i in range(4):
    COPPER_hostname[i] = 'cpr700' + str(i+1)
    COPPER_nodeID[i] = '11744051' + str(i+3)
    eb0_port[i] = '3400' + str(i+1)
    COPPER_shell[i] = Sh_Factory.get_shell(COPPER_hostname[i])
    COPPER_shell[i].sendAndRecieve(des_ser_COPPER_main +" "+ COPPER_hostname[i] +" "+ COPPER_nodeID[i] +" "+ FINNESEbitflag[i] +" "+ UseNSM_bool +" "+ NSMnodeName_cpr)
    KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main +" "+ COPPER_hostname[i] +" "+ UseNSM_bool +" "+ eb0_port[i] +" "+ NSMnodeName_ropc)

#cpr7001 = Sh_Factory.get_shell('cpr7001')
#cpr7002 = Sh_Factory.get_shell('cpr7002')
#cpr7003 = Sh_Factory.get_shell('cpr7003')
#cpr7004 = Sh_Factory.get_shell('cpr7004')
# Usage : RecvSendCOPPER.py <COPPER hostname> <COPPER node ID> <bit flag of FINNESEs> <Use NSM(Network Shared Memory)? yes=1/no=0> <NSM nodename>'
# bit flag of FINESSE
# slot a : 1, slot b : 2, slot c : 4, slot d : 8
# e.g. slot abcd -> bitflag=15, slot bd -> bit flag=10
#cpr7001.sendAndRecieve(des_ser_COPPER_main + " cpr7001 117440513 14 0 1; sleep 3000000;")
#cpr7002.sendAndRecieve(des_ser_COPPER_main + " cpr7002 117440514 15 0 1; sleep 3000000;")
#cpr7003.sendAndRecieve(des_ser_COPPER_main + " cpr7003 117440515 15 0 1; sleep 3000000;")
#cpr7004.sendAndRecieve(des_ser_COPPER_main + " cpr7004 117440516 15 0 1; sleep 3000000;")
#
# basf2 program BEFORE eb0(event builder 0) on a readout PC
#
# Usage : RecvStream0.py <COPPER hostname> <Use NSM(Network Shared Memory)? yes=1/no=0> <port # of eb0> <NSM nodename>
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7001 0 34001 hogehoge1000; sleep 3000000;")
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7002 0 34002 hogehoge1000; sleep 3000000;")
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7003 0 34003 hogehoge1000; sleep 3000000;")
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7004 0 34004 hogehoge1000; sleep 3000000;")


KeyboardInterrupt: 

In [48]:
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/rawdata/src/DesSerPrePC.cc"))
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/rawdata/src/DesSerCOPPER.cc"))
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/ropc/des_ser_COPPER_main.cc"))
#print(KLM01_B.sendAndRecieveRaw("ls /home/group/b2klm/basf2_local/release/daq/copper/daq_scripts/run*"))
print(KLM01_A.copyFrom("/home/group/b2klm/basf2_local/release/daq/copper/daq_scripts/run_start_mono_Barrel_butBF2.sh","run_start_mono_Barrel_butBF2.sh"))

None


## Start the run:
- Where? `@ttd11` (bottom-left terminal, left tab)

- How? launch the following command:
```
trigft -191 <trigger_type> <trigger_rate> [<number_of_events>]
```
    (Launch "trigft" without any arguments to know which arguments the "trigft" command can receive)
    
    
- Check: the `bottom-center terminal`, `left tab` (@ttd11) should pass in a RUNNING state and become `blue` (smile)



## Wait:
- Where? `@klm01` (bottom-right terminal, left tab)
- Check: here the events recorded by each COPPER are shown: within each COPPER the event number must be the same (plus or minus 20 at high rate) and tipically, if N is the event number: N(cpr7001) < N(cpr7002) < N(cpr7003) < N(cpr7004)

# Stop a run


##  Stop the trigger:
- Where? `@ttd11` (bottom-left terminal, left tab)

- How? launch the following commands:
```
resetft -191
```
- Check: the last message shown in the terminal is: `resetting trigger`
- Check: in the `bottom-right terminal`, `left tab` (@klm01) all the numbers go to 0; in the `bottom-center terminal`, `left tab` (@ttd11) many numbers go to 0 (the trigger is correctly reset)



##  Stop the run:
- Where? `@klm01` (top-left terminal, left tab)

- How? launch the following commands:
```
    CTRL+C
    ~/bin/kill_basf2_all.sh
```
- Check: all the small windows are closed, in the terminal is shown the list of all the COPPERs and the last message shown is `Killed`


## Rename the output `.sroot` file:
- Where? `@klm01` (top-left terminal, left tab)

- How? launch the following command:
```
mv root_output.sroot <new_file_name>
```
  - to check naming criteria used to rename the previous runs files use
```
ls | grep .sroot
```
